## Sumário:
- EDA 
- Treinamento
- Validação

## Objetivo
NCJNet é um modelo preditivo que visa sugerir cursos para o usuário, a predição é feito sob análise de notas e preferência do usuário.

## Configuração do Ambiente
É necessário o ambiente virtual para o funcionamento, para isso, realize os seguintes comandos no terminal
- /NCJNet$ pipenv shell
- /NCJNet$ pipenv install

Após isso, o ambiente está configurado com todas as bibliotecas necessárias. Para verificar, compile o código a baixo.

In [115]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Todas as bibliotecas são necessárias para a compilação

# EDA

In [116]:
import pandas as pd
import seaborn as sns

## Criar DataFrames dos Datasets 

In [117]:
# Carregar DataFrame de clientes (customers.csv)
df = pd.read_csv('./datasets/RegistroAcompanhamento.csv') # leitura dos dados do csv

# visualizar os dados dos 5 primeiros clientes
df.head(5)

,Identificador,"""Codigo externo do registro""",Etapa,"""Data da criacao""","""Momento do ultimo ganho""","""Momento da ultima perda""","""Na mesma etapa desde""","""Codigo - Oferta de curso""","""Nome - Oferta de curso""","""Identificador da pessoa""",...,Redacao,Quimica,Fisica,Biologia,Geografia,Historia,Filosofia,Sociologia,Artes,Curso
0,318,"""- - -""","""- - -""","""16/06/2025 20:02""","""- - -""","""- - -""","""- - -""","""- - -""","""- - -""",360,...,90,89,88,86,87,99,86,85,89,Biblioteconomia
1,317,"""- - -""","""- - -""","""16/06/2025 20:00""","""- - -""","""- - -""","""- - -""","""- - -""","""- - -""",359,...,76,76,75,79,78,90,73,80,81,Biblioteconomia
2,316,"""- - -""","""- - -""","""16/06/2025 19:59""","""- - -""","""- - -""","""- - -""","""- - -""","""- - -""",358,...,63,63,61,59,60,83,74,73,76,Biblioteconomia
3,315,"""- - -""","""- - -""","""16/06/2025 19:57""","""- - -""","""- - -""","""- - -""","""- - -""","""- - -""",357,...,69,71,70,69,66,89,68,64,66,Biblioteconomia
4,314,"""- - -""","""- - -""","""16/06/2025 19:55""","""- - -""","""- - -""","""- - -""","""- - -""","""- - -""",356,...,71,89,63,68,87,88,89,90,91,Museologia


In [118]:
# Mostrar estrutura / schema do DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 61 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   Identificador                        300 non-null    object
 1    "Codigo externo do registro"        300 non-null    object
 2    Etapa                               300 non-null    object
 3    "Data da criacao"                   300 non-null    object
 4    "Momento do ultimo ganho"           300 non-null    object
 5    "Momento da ultima perda"           300 non-null    object
 6    "Na mesma etapa desde"              300 non-null    object
 7    "Codigo - Oferta de curso"          300 non-null    object
 8    "Nome - Oferta de curso"            300 non-null    object
 9    "Identificador da pessoa"           300 non-null    int64 
 10   "Codigo externo da pessoa"          300 non-null    object
 11   "Nome da pessoa"                    300 non-

### Selecionar apenas dados necessários

In [119]:
colunas_desejadas = [
	"Matematica",
	'"Portugues "', 
	"Literatura",
	"Redacao",
	"Quimica",
	"Fisica",
	"Biologia",
	"Geografia",
	"Historia",
	"Filosofia",
	"Sociologia",
	"Artes",
	'Area de Preferencia', 
	"Curso"
]

# Remover espaços dos nomes das colunas
df.columns = df.columns.str.strip()

# Verificar nomes das colunas após strip
print(df.columns.tolist())


# Agora filtrar as colunas
df_filtrado = df[colunas_desejadas]
df_filtrado.info()

['Identificador', '"Codigo externo do registro"', 'Etapa', '"Data da criacao"', '"Momento do ultimo ganho"', '"Momento da ultima perda"', '"Na mesma etapa desde"', '"Codigo - Oferta de curso"', '"Nome - Oferta de curso"', '"Identificador da pessoa"', '"Codigo externo da pessoa"', '"Nome da pessoa"', '"Nome do responsavel"', 'Atividade', '"E-mail da pessoa"', '"E-mails secundarios"', '"Telefone da pessoa"', '"Telefones secundarios"', 'CPF', '"Data de nascimento"', '"Valor da oportunidade"', '"Data da atividade"', '"Forma de ingresso"', '"Grau de instrucao"', '"Escola de origem"', '"Nota do ENEM"', '"Processo seletivo"', 'Unidade', 'Modalidade', 'Status', '"Local da oferta"', 'Canal', '"Resumo atual"', 'Objecao', '"Observacoes da perda"', 'Endereco', 'Numero', 'Bairro', 'Cidade', 'UF', 'Concorrentes', 'Probabilidade', '"[AE] Situacao de matricula"', 'Area de Preferencia', '"[AE] % de nota geral"', '"[AE] % de frequencia geral"', '"[AE] Satisfacao"', '"[AE] Qtd de lancamentos Vencidos"', 

In [120]:
df_filtrado.head(5)

,Matematica,"""Portugues """,Literatura,Redacao,Quimica,Fisica,Biologia,Geografia,Historia,Filosofia,Sociologia,Artes,Area de Preferencia,Curso
0,82,96,98,90,89,88,86,87,99,86,85,89,Humanas,Biblioteconomia
1,80,90,89,76,76,75,79,78,90,73,80,81,Humanas,Biblioteconomia
2,65,78,80,63,63,61,59,60,83,74,73,76,Humanas,Biblioteconomia
3,70,90,88,69,71,70,69,66,89,68,64,66,Humanas,Biblioteconomia
4,74,72,69,71,89,63,68,87,88,89,90,91,Humanas,Museologia


In [121]:
#  Medidas estatísticas do DF
df_filtrado.describe()

,Matematica,"""Portugues """,Literatura,Redacao,Quimica,Fisica,Biologia,Geografia,Historia,Filosofia,Sociologia,Artes
count,300.000000,300.000000,300.000000,300.000000,300.0000,300.00000,300.000000,300.000000,300.000000,300.000000,300.00000,300.000000
mean,80.233333,77.190000,77.546667,77.336667,78.4400,79.41000,79.600000,77.813333,78.776667,78.100000,78.35000,78.823333
std,11.578100,9.577735,9.072791,9.430435,10.1609,10.61402,10.555792,8.762864,8.803320,9.097943,9.98096,10.090168
min,58.000000,58.000000,59.000000,48.000000,52.0000,50.00000,43.000000,57.000000,60.000000,59.000000,41.00000,53.000000
25%,69.000000,68.000000,71.000000,70.000000,71.0000,72.00000,72.750000,72.000000,73.750000,73.000000,72.00000,71.000000
50%,80.000000,78.000000,78.000000,77.500000,79.0000,80.00000,80.000000,79.000000,79.000000,79.000000,79.00000,79.000000
75%,90.000000,85.000000,84.000000,84.000000,86.0000,88.25000,88.000000,82.000000,84.000000,84.000000,86.00000,87.000000
max,100.000000,99.000000,99.000000,99.000000,99.0000,99.00000,99.000000,98.000000,99.000000,98.000000,99.00000,99.000000


In [122]:
# Mapeando as áreas de cada curso
df = pd.read_csv('./datasets/RegistroAcompanhamento.csv', encoding='utf-8')
df.columns = df.columns.str.strip()
df['Curso'] = df['Curso'].astype(str).str.strip()
df['Area de Preferencia'] = df['Area de Preferencia'].astype(str).str.strip()

# Corrija encoding dos nomes dos cursos e áreas
df['Curso'] = df['Curso'].apply(lambda x: x.encode('latin1').decode('utf-8') if isinstance(x, str) else x)
df['Area de Preferencia'] = df['Area de Preferencia'].apply(lambda x: x.encode('latin1').decode('utf-8') if isinstance(x, str) else x)

areas_cursos = dict(zip(df['Curso'], df['Area de Preferencia']))

""" areas_cursos = {
    'Sociologia': 'Humanas',
    'Psicologia': 'Humanas',
    'Odontologia': 'Saúde',
    'Medicina Veterinária': 'Biológicas',
    'Medicina': 'Saúde',
    'Engenharia Elétrica': 'Exatas', 
    'Engenharia da Computação': 'Tecnologia',
    'Engenharia Civil': 'Exatas',
    'Enfermagem': 'Saúde',
    'Biologia': 'Biológicas',
    'Ciência da Computação': 'Tecnologia',
    'Arquitetura e Urbanismo': 'Exatas',
    'Direito': 'Humanas',
    'Design': 'Exatas',
    'Artes Visuais': 'Artes',
    'Engenharia de Produção': 'Exatas',
    'Engenharia Mecânica': 'Exatas',
    'Engenharia Química': 'Exatas',
    'Engenharia Ambiental': 'Exatas',
    'Engenharia de Alimentos': 'Exatas',
    'Engenharia de Software': 'Tecnologia',
    'Geografia': 'Humanas',
    'Filosofia': 'Humanas',
    'Química': 'Exatas',
    'Física': 'Exatas',
    'Matemática': 'Exatas',
    'Letras': 'Linguagens',
    'Dança': 'Artes',
    'Música': 'Artes',
    'Teatro': 'Artes',
    'Artes Cênicas': 'Artes',
    'Artes Plásticas': 'Artes',
    'Cinema': 'Artes',
    'Sistemas da Informação': 'Tecnologia',
    'Agronomia': 'Biológicas',
    'História': 'Humanas',
    'Ciências Sociais': 'Humanas',
    'Serviço Social': 'Humanas',
    'Educação Física': 'Saúde',
    'Relações Internacionais': 'Humanas',
    'Jornalismo': 'Comunicação',
    'Publicidade e Propaganda': 'Comunicação',
    'Relações Públicas': 'Comunicação',
    'Marketing': 'Comunicação',
    'Administração': 'Humanas',
    'Farmácia': 'Biológicas',
    'Biomedicina': 'Biológicas',
    'Nutrição': 'Biológicas',
    'Fisioterapia': 'Saúde',
    'Fonoaudiologia': 'Saúde',
    'ADS': 'Tecnologia',
    'Ciência de Dados': 'Tecnologia',
    'Linguística': 'Linguagens',
    'Pedagogia': 'Humanas',
    'Engenharia Florestal': 'Biológicas',
    'Moda': 'Artes',
    'Design Grafico': 'Artes',
    'Design de Interiores': 'Artes',
    'Museologia': 'Humanas',
    'Gestão Ambiental': 'Exatas',
    'Biblioteconomia': 'Humanas'
} """
len(areas_cursos)

60

## Dicionários por área

In [123]:
areas_unicas = df_filtrado['Area de Preferencia'].unique()
utils = {}
modelos_por_area = {
    areas_unicas[0]: {
        "model": None,
        "acc": -float('inf')
        },
    areas_unicas[1]: {
        "model": None,
        "acc": -float('inf')
        },
    areas_unicas[2]: {
        "model": None,
        "acc": -float('inf')
        },
    areas_unicas[3]: {
        "model": None,
        "acc": -float('inf')
        },
    areas_unicas[4]: {
        "model": None,
        "acc": -float('inf')
        },
    areas_unicas[5]: {
        "model": None,
        "acc": -float('inf')
        },
    areas_unicas[6]: {
        "model": None,
        "acc": -float('inf')
        },
    areas_unicas[7]: {
        "model": None,
        "acc": -float('inf')
        }
}

## Features e transformação dos dados

In [124]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import numpy as np
import xgboost as xgb

# Features (X) = notas por disciplina
feature_cols = ['Matematica', '"Portugues "', 'Literatura', 'Redacao', 'Quimica', 'Fisica', 'Biologia', 'Geografia', 'Historia', 'Filosofia', 'Sociologia', 'Artes','Area de Preferencia']

encoder_area_pref_geral = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoder_area_pref_geral.fit(df_filtrado[['Area de Preferencia']]) # Fit no dataset COMPLETO para todas as categorias

scaler = StandardScaler()  # inicializa para normalizar os dados

for area in areas_unicas:
    # Filtrar o DataFrame para a área atual
    df_area = df_filtrado[df_filtrado['Area de Preferencia'] == area].copy()
    
    # Features (X) = notas por disciplina
    X = df_area[feature_cols].copy()
    X_numeric = X.drop(columns=['Area de Preferencia'])
    
    # Rótulo (y) = curso que o aluno faz
    y_categorico = df_area["Curso"]

    # Transformar a variável curso para formato numérico
    label_curso = LabelEncoder()
    y = label_curso.fit_transform(y_categorico)
    
    # Filtrar classes com pelo menos 2 exemplos
    class_counts = y_categorico.value_counts()
    valid_classes = class_counts[class_counts >= 2].index
    df_area = df_area[df_area["Curso"].isin(valid_classes)]

    # Atualizar X e y após o filtro
    X_numeric = df_area[feature_cols].copy()
    X_numeric = X.drop(columns=['Area de Preferencia'])
    y_categorico = df_area["Curso"]
    y = label_curso.fit_transform(y_categorico)
    
    utils[area] = {
        'X': X_numeric,
        'y': y
    }

# Treinamento

## Treinamento com Random Forest

In [125]:
kf_rf = StratifiedKFold(n_splits=5, shuffle=True, random_state=None)
for area in areas_unicas:
    y_labels_rf = []
    y_predict_rf = []

    acc_scores = []

    rf_model = RandomForestClassifier(n_estimators=100, random_state=42, oob_score=True,max_depth=20,criterion='gini')
    best_model = -float('inf')
    print(area)
    # Treinar o modelo Random Forest com validação cruzada
    for train, test in kf_rf.split(utils[area]["X"], utils[area]["y"]):
            
            X_train_fold, X_test_fold = utils[area]["X"].iloc[train], utils[area]["X"].iloc[test] 
            y_train_fold, y_test_fold = utils[area]["y"][train], utils[area]["y"][test] 

            # Normalização dos dados
            X_train_normalizado = scaler.fit_transform(X_train_fold)
            X_test_normalizado = scaler.transform(X_test_fold)
            
            #Treina o modelo com esse conjunto de treinamento
            rf_model.fit(X_train_normalizado, y_train_fold)
            
            # Predições
            y_test_pred = rf_model.predict(X_test_normalizado)

            y_labels_rf.extend(y_test_fold)
            y_predict_rf.extend(y_test_pred)
            acc_scores.append(accuracy_score(y_test_fold, y_test_pred))
            
    if np.max(acc_scores) > modelos_por_area[area]["acc"]:
        modelos_por_area[area]["model"] = rf_model
        modelos_por_area[area]["acc"] = np.max(acc_scores)
        
    # Avaliação do modelo
    print("Avaliação do modelo Random Forest:")
    print(f"Acurácia média: {np.mean(acc_scores):.4f}")
    print(f"Acurácia mínima: {np.min(acc_scores):.4f}")
    print(f"Acurácia máxima: {np.max(acc_scores):.4f}")
    print("Relatório de Classificação:")
    print(classification_report(
        y_labels_rf, y_predict_rf,
        target_names=label_curso.classes_,
        labels=np.arange(len(label_curso.classes_))
    ))
    print("Matriz de Confusão:")
    print(confusion_matrix(
        y_labels_rf, y_predict_rf,
        labels=np.arange(len(label_curso.classes_))
    ))
    print(f"Score OOB (Out-of-Bag) do modelo: {rf_model.oob_score_:.4f}")




 Humanas            


/home/nicolas/.local/share/virtualenvs/NCJNet-xGDSlkFY/lib/python3.11/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  if np.all(self.n_splits > y_counts):


Avaliação do modelo Random Forest:
Acurácia média: 0.4154
Acurácia mínima: 0.3077
Acurácia máxima: 0.5000
Relatório de Classificação:
                           precision    recall  f1-score   support

               Jornalismo       0.57      0.80      0.67         5
                Marketing       0.75      0.60      0.67         5
 Publicidade e Propaganda       0.00      0.00      0.00         4
        Relacoes Publicas       0.20      0.20      0.20         5

                micro avg       0.47      0.42      0.44        19
                macro avg       0.38      0.40      0.38        19
             weighted avg       0.40      0.42      0.40        19

Matriz de Confusão:
[[4 0 0 1]
 [0 3 0 1]
 [0 0 0 0]
 [1 0 1 1]]
Score OOB (Out-of-Bag) do modelo: 0.2353
 Artes              
Avaliação do modelo Random Forest:
Acurácia média: 0.2582
Acurácia mínima: 0.0909
Acurácia máxima: 0.5000
Relatório de Classificação:
                           precision    recall  f1-score   support

/home/nicolas/.local/share/virtualenvs/NCJNet-xGDSlkFY/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  The class to report if `average='binary'` and the data is binary,
/home/nicolas/.local/share/virtualenvs/NCJNet-xGDSlkFY/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  The class to report if `average='binary'` and the data is binary,
/home/nicolas/.local/share/virtualenvs/NCJNet-xGDSlkFY/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
 

Avaliação do modelo Random Forest:
Acurácia média: 0.1679
Acurácia mínima: 0.0000
Acurácia máxima: 0.2857
Relatório de Classificação:
                           precision    recall  f1-score   support

               Jornalismo       0.25      0.40      0.31         5
                Marketing       0.00      0.00      0.00         6
 Publicidade e Propaganda       0.17      0.20      0.18         5
        Relacoes Publicas       0.00      0.00      0.00         5

                micro avg       0.13      0.14      0.14        21
                macro avg       0.10      0.15      0.12        21
             weighted avg       0.10      0.14      0.12        21

Matriz de Confusão:
[[2 0 0 3]
 [1 0 2 0]
 [0 2 1 0]
 [4 0 0 0]]
Score OOB (Out-of-Bag) do modelo: 0.0345
 Saude              
Avaliação do modelo Random Forest:
Acurácia média: 0.1667
Acurácia mínima: 0.0000
Acurácia máxima: 0.3333
Relatório de Classificação:
                           precision    recall  f1-score   support

/home/nicolas/.local/share/virtualenvs/NCJNet-xGDSlkFY/lib/python3.11/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  if np.all(self.n_splits > y_counts):


Avaliação do modelo Random Forest:
Acurácia média: 0.4091
Acurácia mínima: 0.2500
Acurácia máxima: 0.5455
Relatório de Classificação:
                           precision    recall  f1-score   support

               Jornalismo       0.50      0.50      0.50         4
                Marketing       0.33      0.40      0.36         5
 Publicidade e Propaganda       0.25      0.20      0.22         5
        Relacoes Publicas       0.00      0.00      0.00         5

                micro avg       0.25      0.26      0.26        19
                macro avg       0.27      0.28      0.27        19
             weighted avg       0.26      0.26      0.26        19

Matriz de Confusão:
[[2 0 0 0]
 [0 2 0 1]
 [0 0 1 0]
 [0 1 1 0]]
Score OOB (Out-of-Bag) do modelo: 0.3125
 Comunicacao        
Avaliação do modelo Random Forest:
Acurácia média: 0.7100
Acurácia mínima: 0.5000
Acurácia máxima: 1.0000
Relatório de Classificação:
                           precision    recall  f1-score   support

## Treinamento com XGBoost


In [126]:
kf_xgb = StratifiedKFold(n_splits=5, shuffle=True, random_state=None)
for area in areas_unicas:
    y_labels_xgb = []
    y_predict_xgb = []

    acc_scores = []
    # Parametros do modelo XGBoost e número de iterações
    params = {"objective": "multi:softprob", "tree_method": "hist", "num_class": len(np.unique(y))}
    n = 1000

    # Realizar validação cruzada com XGBoost
    # Num_boost_round é o número de iterações
    # Early stopping é usado para evitar overfitting - se não houver melhoria na métrica de validação por 50 iterações, o treinamento é interrompido
    # Nfold é o número de divisões para validação cruzada
    # Params são os parâmetros do modelo
    # Metrics são as métricas a serem avaliadas durante a validação cruzada
    xgb_model = xgb.XGBClassifier(
    objective="multi:softmax",
    tree_method="hist",
    num_class=len(np.unique(y)),
    n_estimators=n,
    eval_metric="mlogloss",
    verbosity=1,
    random_state=42,
    )
    print(area)
    # Treinar o modelo Random Forest com validação cruzada
    for train, test in kf_xgb.split(utils[area]["X"],utils[area]["y"]):
            
            X_train_fold, X_test_fold = utils[area]["X"].iloc[train], utils[area]["X"].iloc[test] 
            y_train_fold, y_test_fold = utils[area]["y"][train], utils[area]["y"][test] 

            # Normalização dos dados
            X_train_normalizado = scaler.fit_transform(X_train_fold)
            X_test_normalizado = scaler.transform(X_test_fold)

            #Treina o modelo com esse conjunto de treinamento
            xgb_model.fit(X_train_normalizado, y_train_fold)
            
            # Predições
            y_test_pred = xgb_model.predict(X_test_normalizado)

            y_labels_xgb.extend(y_test_fold)
            y_predict_xgb.extend(y_test_pred)
            acc_scores.append(accuracy_score(y_test_fold, y_test_pred))

    if np.max(acc_scores) > modelos_por_area[area]["acc"]:
        modelos_por_area[area]['model'] = rf_model
        modelos_por_area[area]["acc"] = np.max(acc_scores)

    print("Avaliação do modelo XGBoost:")
    print(f"Acurácia média: {np.mean(acc_scores):.4f}")
    print(f"Acurácia mínima: {np.min(acc_scores):.4f}")
    print(f"Acurácia máxima: {np.max(acc_scores):.4f}")
    print("Relatório de Classificação:")
    print(classification_report(
        y_labels_xgb, y_predict_xgb,
        target_names=label_curso.classes_,
        labels=np.arange(len(label_curso.classes_))
    ))
    print("Matriz de Confusão:")
    print(confusion_matrix(
        y_labels_xgb, y_predict_xgb,
        labels=np.arange(len(label_curso.classes_))
    ))

 Humanas            


/home/nicolas/.local/share/virtualenvs/NCJNet-xGDSlkFY/lib/python3.11/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  if np.all(self.n_splits > y_counts):


Avaliação do modelo XGBoost:
Acurácia média: 0.3064
Acurácia mínima: 0.1538
Acurácia máxima: 0.5000
Relatório de Classificação:
                           precision    recall  f1-score   support

               Jornalismo       0.60      0.60      0.60         5
                Marketing       0.60      0.60      0.60         5
 Publicidade e Propaganda       0.00      0.00      0.00         4
        Relacoes Publicas       0.50      0.20      0.29         5

                micro avg       0.41      0.37      0.39        19
                macro avg       0.42      0.35      0.37        19
             weighted avg       0.45      0.37      0.39        19

Matriz de Confusão:
[[3 0 0 0]
 [0 3 0 0]
 [0 0 0 0]
 [0 1 1 1]]
 Artes              
Avaliação do modelo XGBoost:
Acurácia média: 0.2764
Acurácia mínima: 0.1000
Acurácia máxima: 0.5000
Relatório de Classificação:
                           precision    recall  f1-score   support

               Jornalismo       0.25      0.20     

/home/nicolas/.local/share/virtualenvs/NCJNet-xGDSlkFY/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  The class to report if `average='binary'` and the data is binary,
/home/nicolas/.local/share/virtualenvs/NCJNet-xGDSlkFY/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  The class to report if `average='binary'` and the data is binary,
/home/nicolas/.local/share/virtualenvs/NCJNet-xGDSlkFY/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
 

Avaliação do modelo XGBoost:
Acurácia média: 0.1929
Acurácia mínima: 0.0000
Acurácia máxima: 0.5714
Relatório de Classificação:
                           precision    recall  f1-score   support

               Jornalismo       0.00      0.00      0.00         5
                Marketing       0.33      0.33      0.33         6
 Publicidade e Propaganda       0.17      0.20      0.18         5
        Relacoes Publicas       0.00      0.00      0.00         5

                micro avg       0.14      0.14      0.14        21
                macro avg       0.12      0.13      0.13        21
             weighted avg       0.13      0.14      0.14        21

Matriz de Confusão:
[[0 2 0 2]
 [1 2 1 0]
 [1 1 1 0]
 [4 0 1 0]]
 Saude              
Avaliação do modelo XGBoost:
Acurácia média: 0.2667
Acurácia mínima: 0.1667
Acurácia máxima: 0.3333
Relatório de Classificação:
                           precision    recall  f1-score   support

               Jornalismo       0.50      0.60     

/home/nicolas/.local/share/virtualenvs/NCJNet-xGDSlkFY/lib/python3.11/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  if np.all(self.n_splits > y_counts):


Avaliação do modelo XGBoost:
Acurácia média: 0.3045
Acurácia mínima: 0.2500
Acurácia máxima: 0.4167
Relatório de Classificação:
                           precision    recall  f1-score   support

               Jornalismo       0.50      0.25      0.33         4
                Marketing       0.60      0.60      0.60         5
 Publicidade e Propaganda       0.14      0.20      0.17         5
        Relacoes Publicas       0.00      0.00      0.00         5

                micro avg       0.26      0.26      0.26        19
                macro avg       0.31      0.26      0.28        19
             weighted avg       0.30      0.26      0.27        19

Matriz de Confusão:
[[1 0 1 0]
 [0 3 1 0]
 [0 0 1 1]
 [0 0 1 0]]
 Comunicacao        
Avaliação do modelo XGBoost:
Acurácia média: 0.8100
Acurácia mínima: 0.5000
Acurácia máxima: 1.0000
Relatório de Classificação:
                           precision    recall  f1-score   support

               Jornalismo       1.00      0.60     

## Treinamento com SVM

In [127]:
kf_svm = StratifiedKFold(n_splits=5, shuffle=True, random_state=None)

for area in areas_unicas:
    
    y_labels_svm = []
    y_predict_svm = []

    acc_scores = []

    # Criar o modelo SVM com kernel RBF
    svm_model = SVC(kernel='rbf', C=1.0, gamma='scale')

    print(area)

    # Treinar o modelo SVM com validação cruzada
    for train, test in kf_svm.split(utils[area]["X"], utils[area]["y"]):
        
        X_train_fold, X_test_fold = utils[area]["X"].iloc[train], utils[area]["X"].iloc[test] 
        y_train_fold, y_test_fold = utils[area]["y"][train], utils[area]["y"][test] 
        
        # Treina o modelo com o conjunto de treinamento
        svm_model.fit(X_train_fold, y_train_fold)
        y_test_pred = svm_model.predict(X_test_fold)
        
        y_labels_svm.extend(y_test_fold)
        y_predict_svm.extend(y_test_pred)
        acc_scores.append(accuracy_score(y_test_fold, y_test_pred))
        
    if np.max(acc_scores) > modelos_por_area[area]["acc"]:
        modelos_por_area[area]["model"] = rf_model
        modelos_por_area[area]["acc"] = np.max(acc_scores)

    print("Avaliação do modelo SVM:")
    print(f"Acurácia média: {np.mean(acc_scores):.4f}")
    print(f"Acurácia mínima: {np.min(acc_scores):.4f}")
    print(f"Acurácia máxima: {np.max(acc_scores):.4f}")
    print("Relatório de Classificação:")
    print(classification_report(
        y_labels_svm, y_predict_svm,
        target_names=label_curso.classes_,
        labels=np.arange(len(label_curso.classes_))
    ))
    print("Matriz de Confusão:")
    print(confusion_matrix(
        y_labels_svm, y_predict_svm,
        labels=np.arange(len(label_curso.classes_))
    ))


 Humanas            
Avaliação do modelo SVM:
Acurácia média: 0.3462
Acurácia mínima: 0.1667
Acurácia máxima: 0.5385
Relatório de Classificação:
                           precision    recall  f1-score   support

               Jornalismo       0.50      0.40      0.44         5
                Marketing       0.67      0.40      0.50         5
 Publicidade e Propaganda       0.00      0.00      0.00         4
        Relacoes Publicas       0.40      0.40      0.40         5

                micro avg       0.46      0.32      0.38        19
                macro avg       0.39      0.30      0.34        19
             weighted avg       0.41      0.32      0.35        19

Matriz de Confusão:
[[2 0 0 0]
 [0 2 0 1]
 [1 0 0 0]
 [0 0 0 2]]
 Artes              


/home/nicolas/.local/share/virtualenvs/NCJNet-xGDSlkFY/lib/python3.11/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  if np.all(self.n_splits > y_counts):


Avaliação do modelo SVM:
Acurácia média: 0.2164
Acurácia mínima: 0.1000
Acurácia máxima: 0.3000
Relatório de Classificação:
                           precision    recall  f1-score   support

               Jornalismo       0.00      0.00      0.00         5
                Marketing       0.08      0.20      0.12         5
 Publicidade e Propaganda       0.00      0.00      0.00         5
        Relacoes Publicas       0.50      0.40      0.44         5

                micro avg       0.16      0.15      0.15        20
                macro avg       0.15      0.15      0.14        20
             weighted avg       0.15      0.15      0.14        20

Matriz de Confusão:
[[0 0 0 0]
 [0 1 1 0]
 [0 2 0 1]
 [0 0 0 2]]
 Linguagens         
Avaliação do modelo SVM:
Acurácia média: 0.6000
Acurácia mínima: 0.0000
Acurácia máxima: 1.0000
Relatório de Classificação:
                           precision    recall  f1-score   support

               Jornalismo       0.57      0.80      0.67   

/home/nicolas/.local/share/virtualenvs/NCJNet-xGDSlkFY/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  The class to report if `average='binary'` and the data is binary,
/home/nicolas/.local/share/virtualenvs/NCJNet-xGDSlkFY/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  The class to report if `average='binary'` and the data is binary,
/home/nicolas/.local/share/virtualenvs/NCJNet-xGDSlkFY/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
 

                           precision    recall  f1-score   support

               Jornalismo       0.25      0.20      0.22         5
                Marketing       0.00      0.00      0.00         5
 Publicidade e Propaganda       0.20      0.20      0.20         5
        Relacoes Publicas       0.75      0.60      0.67         5

                micro avg       0.22      0.25      0.23        20
                macro avg       0.30      0.25      0.27        20
             weighted avg       0.30      0.25      0.27        20

Matriz de Confusão:
[[1 2 0 0]
 [2 0 0 0]
 [0 2 1 1]
 [0 0 2 3]]
 Tecnologia         
Avaliação do modelo SVM:
Acurácia média: 0.1333
Acurácia mínima: 0.0000
Acurácia máxima: 0.1667
Relatório de Classificação:
                           precision    recall  f1-score   support

               Jornalismo       0.00      0.00      0.00         5
                Marketing       0.00      0.00      0.00         5
 Publicidade e Propaganda       0.12      0.20   

/home/nicolas/.local/share/virtualenvs/NCJNet-xGDSlkFY/lib/python3.11/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  if np.all(self.n_splits > y_counts):


Avaliação do modelo SVM:
Acurácia média: 0.7100
Acurácia mínima: 0.5000
Acurácia máxima: 1.0000
Relatório de Classificação:
                           precision    recall  f1-score   support

               Jornalismo       1.00      0.60      0.75         5
                Marketing       0.75      0.60      0.67         5
 Publicidade e Propaganda       0.55      1.00      0.71         6
        Relacoes Publicas       1.00      0.60      0.75         5

                 accuracy                           0.71        21
                macro avg       0.82      0.70      0.72        21
             weighted avg       0.81      0.71      0.72        21

Matriz de Confusão:
[[3 0 2 0]
 [0 3 2 0]
 [0 0 6 0]
 [0 1 1 3]]


## Melhores modelos

In [128]:
for area in areas_unicas:
    print(f"Área: {area}")
    print(f"Melhor modelo: {modelos_por_area[area]['model'].__class__.__name__}")
    print(f"Acurácia: {modelos_por_area[area]['acc']:.4f}")
    print()

Área:  Humanas            
Melhor modelo: RandomForestClassifier
Acurácia: 0.5385

Área:  Artes              
Melhor modelo: RandomForestClassifier
Acurácia: 0.5000

Área:  Linguagens         
Melhor modelo: RandomForestClassifier
Acurácia: 1.0000

Área:  Biologicas         
Melhor modelo: RandomForestClassifier
Acurácia: 0.5714

Área:  Saude              
Melhor modelo: RandomForestClassifier
Acurácia: 0.5000

Área:  Tecnologia         
Melhor modelo: RandomForestClassifier
Acurácia: 0.3333

Área:  Exatas             
Melhor modelo: RandomForestClassifier
Acurácia: 0.5455

Área:  Comunicacao        
Melhor modelo: RandomForestClassifier
Acurácia: 1.0000



## Salvar modelos

In [129]:
import joblib 

for area in areas_unicas:
    joblib.dump(modelos_por_area[area]['model'], f'./model/{area}_sugestor.pkl')

# salvar encoder
joblib.dump(label_curso, './model/labelCurso.pkl')
joblib.dump(areas_cursos, './model/areaCursos.pkl')

['./model/areaCursos.pkl']